# Predictive Thyroid Disease Project - Modeling

To make a predictive model for thyroid cancer patients to see how likely after treatment their cancer will reoccur.

'Response to treatment' is considered if the cancer comes back or not

# Initial Set Up

In [1]:
# General Data Manipulation
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


# Import my custom library 
import MyCustDataSciLib as MyCustDataSciLib

In [2]:
# Load training CSV file
df_train = pd.read_csv('4_Thyroid_Disease_FeatureEngineer_train.csv')

df_test = pd.read_csv('4_Thyroid_Disease_FeatureEngineer_test.csv') 

In [3]:
df_train.head()

,age,age_group,gender,smoking,hx_smoking,hx_radiotherapy,thyroid_function,thyroid_function_3cat,thyroid_function_2cat,physical_examination,...,t_4cat,n,m,stage,stage_4cat,stage_2cat,response,response_3cat,recurrence,kmode_cluster_3cat
0,40,40s,f,no,no,no,euthyroid,normal,normal,multinodular_goiter,...,t2,n0,m0,i,i,early,excellent,excellent,no,1
1,62,60s,f,no,no,no,euthyroid,normal,normal,multinodular_goiter,...,t3,n1b,m0,ii,ii,early,structural_incomplete,negative,yes,2
2,29,20s,f,no,no,no,euthyroid,normal,normal,multinodular_goiter,...,t2,n1a,m0,i,i,early,indeterminate,indeterminate,no,1
3,37,30s,f,no,no,no,euthyroid,normal,normal,single_nodular_goiter-left,...,t2,n0,m0,i,i,early,excellent,excellent,no,1
4,31,30s,f,no,no,no,euthyroid,normal,normal,single_nodular_goiter-left,...,t2,n1b,m0,i,i,early,structural_incomplete,negative,yes,2


In [4]:
df_test.head()

,age,age_group,gender,smoking,hx_smoking,hx_radiotherapy,thyroid_function,thyroid_function_3cat,thyroid_function_2cat,physical_examination,...,t_4cat,n,m,stage,stage_4cat,stage_2cat,response,response_3cat,recurrence,kmode_cluster_3cat
0,32,30s,f,no,no,no,euthyroid,normal,normal,single_nodular_goiter-left,...,t3,n0,m0,i,i,early,excellent,excellent,no,0
1,30,30s,f,no,no,no,subclinical_hypothyroidism,subclinical,diseased,normal,...,t3,n0,m0,i,i,early,excellent,excellent,no,0
2,30,30s,f,no,no,no,euthyroid,normal,normal,multinodular_goiter,...,t3,n1b,m0,i,i,early,structural_incomplete,negative,yes,2
3,51,50s,f,no,no,no,euthyroid,normal,normal,single_nodular_goiter-left,...,t3,n0,m0,i,i,early,structural_incomplete,negative,yes,2
4,43,40s,f,no,yes,no,euthyroid,normal,normal,single_nodular_goiter-right,...,t1,n0,m0,i,i,early,excellent,excellent,no,1


In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [10]:
# Define features and target
X_train = df_train.drop('recurrence', axis=1)
y_train = df_train['recurrence']
X_test = df_test.drop('recurrence', axis=1)
y_test = df_test['recurrence']

In [ ]:
df_test.info()

# Model 1 

In [ ]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))